# Transit model test 3 alone

Just pulled in from the other notebook to be able to work on it directly.

Make sure you update your configfile accordingly:

```ini
[setup]
data_set = simple_transit
```

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.constants import G
import astropy.units as u

from sherpa.models import model
from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.fit import Fit
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2
from sherpa.plot import FitPlot

os.chdir('../../../exotic-ism')
import margmodule as marg
from config import CONFIG_INI

In [ ]:
# Test parameters
planet_sys = CONFIG_INI.get('setup', 'data_set')
dtosec = CONFIG_INI.getfloat('constants', 'dtosec')
period = CONFIG_INI.getfloat(planet_sys, 'Per')
Per = period * dtosec
aor = CONFIG_INI.getfloat(planet_sys, 'aor')
constant1 = (G * Per * Per / (4 *np.pi * np.pi))**(1/3)
msmpr = (aor/(constant1))**3
print('msmpr: {}'.format(msmpr))
print('G: {}'.format(G.value))
print('Per: {} sec'.format(Per))

# limb darkening parameters
c1 = 0.66396105
c2 = -0.12617095
c3 = 0.053649047
c4 = -0.026713433

# Create x-array for phase - careful, this is not a regular grid, but consists of three groups of data points
data_x = np.array([-0.046, -0.044, -0.042, -0.040, -0.038, -0.036, -0.034,
                   -0.032, -0.030, -0.006, -0.004, -0.002, 0.0, 0.002, 0.004,
                   0.006, 0.008, 0.01, 0.032, 0.034, 0.036, 0.038, 0.040,
                   0.042, 0.044, 0.046,0.048])

# Make denser and REGULAR x grid for plotting of smooth model
smooth_x = np.arange(data_x[0], data_x[-1], 0.001)

# Initial flux data
original_y = np.array([1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.99000000,
                   0.99000000, 0.99000000, 0.99000000, 0.99000000, 0.99000000,
                   0.99000000, 0.99000000, 0.99000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000])

uncertainty = np.array([0.0004] * len(data_x))

random_scatter = np.array([0.32558253, -0.55610514, -1.1150768, -1.2337022, -1.2678875,
                           0.60321692, 1.1025507, 1.5080730, 0.76113001, 0.51978011,
                           0.72241364, -0.086782108, -0.22698337, 0.22780245, 0.47119014,
                           -2.1660677, -1.2477670, 0.28568456, 0.40292731, 0.077955817,
                           -1.1090623, 0.66895172, -0.59215439, 0.79973968, 1.0603756,
                           0.82684954, -1.8334587])
print('random_scatter.shape: {}'.format(random_scatter.shape))

data_y_no_slope = original_y + (random_scatter * uncertainty)

# Create linear slope
m_fac = 0.04
line = (data_x * m_fac) + 1.00

# add the systematic model (slope/line) to the y array
data_y = line * data_y_no_slope

In [ ]:
# Quick visualization of data
plt.scatter(data_x, data_y)
plt.plot(data_x, line, c='r')
plt.ylim(0.987, 1.004)

In [ ]:
# Make Sherpa data object out of this
data = Data1D('example_transit', data_x, data_y, staterror=uncertainty)   # create data object
dplot = DataPlot()         # create data *plot* object
dplot.prepare(data)   # prepare plot
dplot.plot()

In [ ]:
# Create and visualize model
model = marg.Transit(data_x[0], msmpr, c1, c2, c3, c4, flux0=data_y[0], x_in_phase=True, name='transit', sh=None)
print(model)

In [ ]:
# Freeze almost all parameters
# Note how m_fac stays thawed
model.flux0.freeze()
model.epoch.freeze()
model.inclin.freeze()
model.msmpr.freeze()
model.ecc.freeze()
model.hstp1.freeze()
model.hstp2.freeze()
model.hstp3.freeze()
model.hstp4.freeze()
model.xshift1.freeze()
model.xshift2.freeze()
model.xshift3.freeze()
model.xshift4.freeze()

print(model)

In [ ]:
# Set up statistics and optimizer
stat = Chi2()
opt = LevMar()
opt.config['epsfcn'] = np.finfo(float).eps   # adjusting epsfcn to double precision
#print(stat)
print(opt)

# Set up fit
tfit = Fit(data, model, stat=stat, method=opt)
print('Fit information:')
print(tfit)

In [ ]:
# Perform the fit
fitresult = tfit.fit()
print(fitresult)

### Fit results

In [ ]:
# smooth model
b0_smooth = marg.impact_param((model.period.val*u.d).to(u.s), model.msmpr.val, smooth_x, model.inclin.val*u.rad)
smooth_y, _mulimbf2 = marg.occultnl(model.rl.val, model.c1.val, model.c2.val, model.c3.val, model.c4.val, b0_smooth)

In [ ]:
# Errors from Hessian
calc_errors = np.sqrt(fitresult.extra_output['covar'].diagonal())
rl_err = calc_errors[0]
m_fac_err = calc_errors[1]

# Results
print('rl = {} +/- {}'.format(model.rl.val, rl_err))
print('m_fac = {} +/- {}'.format(model.m_fac.val, m_fac_err))
print('Reduced chi-squared: {}'.format(fitresult.rstat))

# Display model from denser grid over real data after fitting
plt.figure(figsize=(11, 5))
plt.plot(smooth_x, smooth_y, c='orange', label='smooth model')
plt.errorbar(data_x, data_y, yerr=uncertainty, fmt='.', label='data')
plt.plot(data_x, line, c='r', linestyle='dashed', label='systematic model')
plt.legend()
plt.xlabel('phase')
plt.ylabel('flux')
plt.title('Smooth model test 3 over actual data')